## Lesson 13 - Graphs with Python and Neo4j





### Table of Contents

* [py2neo](#py2neo)
* [Cypher query and fetch data](#fetch_data)
* [Delete node on Neo4j server](#delete_node)
* [Create Nodes and Relationship again](#create_node_again)
* [Object Graph Mapping](#ogm)
* [Create node by Person class](#class)
* [MERGE AND UNIQUE Nodes](#merge)



<a id="py2neo"></a>
## py2neo

py2neo is one of Neo4j's Python drivers. It offers a fully-featured interface for interacting with your data in Neo4j. Install py2neo with pip install py2neo.

```python
pip install py2neo==5.0b1
```

```python
pip install py2neo
```
https://py2neo.org/

In [1]:
from py2neo import Graph

# under 4.2.9
# http://127.0.0.1:7474/browser/
graph = Graph('http://127.0.0.1:7474', username='neo4j', password='42840667')

# above 4.3+
# pip uninstall py2neo
# graph = Graph("bolt://localhost:7687", auth=("neo4j", "42840667"))

### playground

- Nodes - graph data records
- Relationships - connect nodes
- Properties - named data values

<img src="images/cypher_graph_v1.jpg">

Ref. https://py2neo.org/v3/types.html

In [2]:
'''
### Node & relationship
'''

from py2neo import Node, Relationship
a = Node('Person', name='David')
b = Node('Person', name='Arnold')
ab = Relationship(a, 'KNOWS', b)
print(a, b, ab)

(:Person {name: 'David'}) (:Person {name: 'Arnold'}) (David)-[:KNOWS {}]->(Arnold)


<img src="images/Person.jpg">

In [3]:
# add properties
a['age'] = 35
b['age'] = 28
ab['time'] = '2021/07/03'
print(a, b, ab)

(:Person {age: 35, name: 'David'}) (:Person {age: 28, name: 'Arnold'}) (David)-[:KNOWS {time: '2021/07/03'}]->(Arnold)


In [4]:
# set default properties if node doesn't have that properties
a.setdefault('location', 'Taipei')
print(a, b)

(:Person {age: 35, location: 'Taipei', name: 'David'}) (:Person {age: 28, name: 'Arnold'})


In [5]:
b.setdefault('gender','unknown')
print(a, b)

(:Person {age: 35, location: 'Taipei', name: 'David'}) (:Person {age: 28, gender: 'unknown', name: 'Arnold'})


### Update node

In [6]:
# update node
data = {
    'name': 'Gracia',
    'age': 24
}
a.update(data)
print(a)

(:Person {age: 24, location: 'Taipei', name: 'Gracia'})


In [7]:
print(a, b, ab)

(:Person {age: 24, location: 'Taipei', name: 'Gracia'}) (:Person {age: 28, gender: 'unknown', name: 'Arnold'}) (Gracia)-[:KNOWS {time: '2021/07/03'}]->(Arnold)


In [8]:
# delete properties, but properties doesn't exist
del a['age']
print(a["name"])

Gracia


In [9]:
# update properties and append relationship
a['age']=20
ab['time']='2021/07/02'

In [10]:
# update relationship, and add properties to node
a.update(age=22, sex='female')
ab.update(time='2021/09/03')

In [11]:
print(a, b, ab)

(:Person {age: 22, location: 'Taipei', name: 'Gracia', sex: 'female'}) (:Person {age: 28, gender: 'unknown', name: 'Arnold'}) (Gracia)-[:KNOWS {time: '2021/09/03'}]->(Arnold)


### Subgraphs

A Subgraph is a collection of nodes and relationships. The simplest way to construct a subgraph is by combining nodes and relationships using standard set operations. For example:

In [12]:
from py2neo import Graph, Node, Relationship

a = Node('Person', name='Alice', age=21, location='台北')
b = Node('Person', name='Bob', age=22, location='台中')
c = Node('Person', name='Mike', age=23, location='高雄')
ab = Relationship(a, 'ACTED_IN', b)
bc = Relationship(b, 'KNOWS', c)
# graph.create(a)
# graph.create(ab)
# graph.create(bc)
subgraph = a | ab | bc
print(subgraph)

Subgraph({Node('Person', age=23, location='高雄', name='Mike'), Node('Person', age=21, location='台北', name='Alice'), Node('Person', age=22, location='台中', name='Bob')}, {KNOWS(Node('Person', age=22, location='台中', name='Bob'), Node('Person', age=23, location='高雄', name='Mike')), ACTED_IN(Node('Person', age=21, location='台北', name='Alice'), Node('Person', age=22, location='台中', name='Bob'))})


In [13]:
subgraph.labels

frozenset({'Person'})

In [14]:
subgraph.keys()

frozenset({'age', 'location', 'name'})

In [15]:
subgraph.types()

frozenset({'ACTED_IN', 'KNOWS'})

In [16]:
for item in subgraph.nodes:
    print('subgraph node:', item)

subgraph node: (:Person {age: 23, location: '\u9ad8\u96c4', name: 'Mike'})
subgraph node: (:Person {age: 21, location: '\u53f0\u5317', name: 'Alice'})
subgraph node: (:Person {age: 22, location: '\u53f0\u4e2d', name: 'Bob'})


### connect to neo4j server and create nodes

### Create subgraph

```
graph.create(subgraph)
```

In [17]:
graph.create(subgraph)

<img src="images/py2neo_playground_subgraph.png">

### Find one user with graph.match

In [18]:
from py2neo import Graph, NodeMatcher

persons = graph.nodes.match("Person", name="Alice").first()
print(persons)

(_1:Person {age: 21, location: '\u53f0\u5317', name: 'Alice'})


In [19]:
persons = graph.nodes.match("Person", age=21)
for person in persons:
    print(person.end_node)

(_1:Person {age: 21, location: '\u53f0\u5317', name: 'Alice'})


### Add node to database

In [20]:
d = Node('Person', name='Billy', age=23, location='花蓮')
ad = Relationship(a, 'KNOWS', d)
# graph.create(d)
# graph.create(ad)
subgraph = ad
graph.create(subgraph)

In [21]:
## Search nodes if Billy exists
name = 'Billy'
if len(graph.nodes.match("Person", name="Billy"))>0:
    print(f"{name} exist")

Billy exist


### LIKE match

In [22]:
list(graph.nodes.match("Person").where("_.name =~ 'B.*'"))

[Node('Person', age=22, location='台中', name='Bob'),
 Node('Person', age=23, location='花蓮', name='Billy')]

In [23]:
list(graph.nodes.match("Person").where("_.name =~ 'M.*'").order_by("_.name").limit(3))

[Node('Person', age=23, location='高雄', name='Mike')]

In [24]:
len(graph.nodes.match("Person").where("_.name =~ 'M.*'"))

1

In [25]:
keanu = graph.nodes.match("Person", name="Alice").first()
#list(graph.relationships.match((keanu, None), "ACTED_IN").limit(3))
for person in list(graph.relationships.match((keanu, None), "ACTED_IN")):
    print(person)

(Alice)-[:ACTED_IN {}]->(Bob)


### NodeMatcher and RelationshipMatcher

In [26]:
import py2neo
from py2neo import Graph, Node, Relationship, NodeMatcher, RelationshipMatcher
print(py2neo.__version__)

5.0b1


In [27]:
matcher = NodeMatcher(graph)
# match.where("_.age >= 30")
nodeList = list(matcher.match().where("_.name =~ 'B.*'").order_by("_.age").limit(4))
print(len(nodeList))
for i in nodeList:
    print(i, i["name"])

2
(_2:Person {age: 22, location: '\u53f0\u4e2d', name: 'Bob'}) Bob
(_3:Person {age: 23, location: '\u82b1\u84ee', name: 'Billy'}) Billy


In [28]:
rmatcher = RelationshipMatcher(graph)
nodeList = rmatcher.match(nodes=(a,b), r_type="ACTED_IN")
print(len(nodeList))
for i in nodeList:
    print(i)

1
(Alice)-[:ACTED_IN {}]->(Bob)


In [29]:
nodeList = rmatcher.match(nodes=(a,b), r_type="ACTED_IN")
print(len(nodeList))
for i in nodeList:
    print(i)
    print(i.start_node)
    print(i.start_node.identity)
    print(i.end_node)
    print(i.end_node.identity)
    print(i.relationships)
    print(i.labels)
    for node in i.nodes:
        print(node)
    print(i.types)

1
(Alice)-[:ACTED_IN {}]->(Bob)
(_1:Person {age: 21, location: '\u53f0\u5317', name: 'Alice'})
1
(_2:Person {age: 22, location: '\u53f0\u4e2d', name: 'Bob'})
2
(ACTED_IN(Node('Person', age=21, location='台北', name='Alice'), Node('Person', age=22, location='台中', name='Bob')),)
frozenset({'Person'})
(_1:Person {age: 21, location: '\u53f0\u5317', name: 'Alice'})
(_2:Person {age: 22, location: '\u53f0\u4e2d', name: 'Bob'})
<bound method Subgraph.types of ACTED_IN(Node('Person', age=21, location='台北', name='Alice'), Node('Person', age=22, location='台中', name='Bob'))>


In [30]:
nodeList = rmatcher.match(nodes=(a,b))
print(len(nodeList))
for i in nodeList:
    print(i)
    print(i.start_node)
    print(i.start_node.identity)
    print(i.end_node)
    print(i.end_node.identity)
    print(i.relationships)
    print(i.labels)
    for node in i.nodes:
        print(node)
    print(i.types)

1
(Alice)-[:ACTED_IN {}]->(Bob)
(_1:Person {age: 21, location: '\u53f0\u5317', name: 'Alice'})
1
(_2:Person {age: 22, location: '\u53f0\u4e2d', name: 'Bob'})
2
(ACTED_IN(Node('Person', age=21, location='台北', name='Alice'), Node('Person', age=22, location='台中', name='Bob')),)
frozenset({'Person'})
(_1:Person {age: 21, location: '\u53f0\u5317', name: 'Alice'})
(_2:Person {age: 22, location: '\u53f0\u4e2d', name: 'Bob'})
<bound method Subgraph.types of ACTED_IN(Node('Person', age=21, location='台北', name='Alice'), Node('Person', age=22, location='台中', name='Bob'))>


In [31]:
# 不存在的 relationship
nodeList = rmatcher.match(nodes=(a,c), r_type="ACTED_IN")
print(len(nodeList))

0


In [32]:
# 搜尋start_node為a的 relationship 有哪些node
nodeList = rmatcher.match(nodes=(a,), r_type="ACTED_IN")
print(len(nodeList))
for i in nodeList:
    print(i)
    print(i.start_node)
    print(i.start_node.identity)
    print(i.end_node)
    print(i.end_node.identity)
    print(i.relationships)
    print(i.labels)
    for node in i.nodes:
        print(node)
    print(i.types)

1
(Alice)-[:ACTED_IN {}]->(Bob)
(_1:Person {age: 21, location: '\u53f0\u5317', name: 'Alice'})
1
(_2:Person {age: 22, location: '\u53f0\u4e2d', name: 'Bob'})
2
(ACTED_IN(Node('Person', age=21, location='台北', name='Alice'), Node('Person', age=22, location='台中', name='Bob')),)
frozenset({'Person'})
(_1:Person {age: 21, location: '\u53f0\u5317', name: 'Alice'})
(_2:Person {age: 22, location: '\u53f0\u4e2d', name: 'Bob'})
<bound method Subgraph.types of ACTED_IN(Node('Person', age=21, location='台北', name='Alice'), Node('Person', age=22, location='台中', name='Bob'))>


### Update node on neo4j server

In [34]:
from py2neo import Graph, Node

person = graph.nodes.match("Person", name="Alice")

for node in person:
    node['gender'] = "female"
    graph.push(node)

<a id="fetch_data"></a>
## Py2neo cypher query and fetch data

In [35]:
# MERGE (a0:Foo {name:"bar"}) SET a0:Foo SET a0+={name: 'bar'} RETURN a0 LIMIT 1
dt = graph.run("MATCH (a:Person) RETURN a.name, a.age, a.location LIMIT 10").to_table()
dt

a.name,a.age,a.location
Mike,23,高雄
Alice,21,台北
Bob,22,台中
Billy,23,花蓮


In [36]:
ndarray = graph.run("MATCH (a:Person) RETURN a.name, a.age, a.location LIMIT 10").to_ndarray()
ndarray

array([['Mike', '23', '高雄'],
       ['Alice', '21', '台北'],
       ['Bob', '22', '台中'],
       ['Billy', '23', '花蓮']], dtype='<U5')

In [37]:
ndf = graph.run("MATCH (a:Person) RETURN a.name, a.age, a.location LIMIT 10").to_data_frame()
ndf

,a.name,a.age,a.location
0,Mike,23,高雄
1,Alice,21,台北
2,Bob,22,台中
3,Billy,23,花蓮


<a id="delete_node"></a>
## Delete node with Person on Neo4j server

<font color=red>Notice! this step will drop all Person nodes.</font>

In [38]:
graph.run("MATCH (n:`Person`) OPTIONAL MATCH (n)-[r]-() DELETE r,n")

### create, delete neo4j database
py2neo operates the neo4j database through the graph object. The current neo4j only supports one database to define a graph

Complete the connection to the database through the initialization function of Graph and create a graph object

graph.create() can write subgraphs to the database or only one node or relationship at a time

graph.delete() deletes the specified subgraph, graph.delete_all() deletes all subgraphs

graph.seperate() deletes the specified relationship

In [ ]:
# DELETE relationships
# graph.separate(ac)

In [39]:
# DELETE ALL GRAPH
graph.delete_all()

<a id="create_node_again"></a>
## Create Nodes and Relationship again

In [ ]:
from py2neo import Graph, Node, Relationship

a = Node('Person', name='Alice', age=21, location='台北')
b = Node('Person', name='Bob', age=22, location='台中')
c = Node('Person', name='Mike', age=23, location='高雄')
d = Node('Person', name='Billy', age=23, location='花蓮')
ab = Relationship(a, 'ACTED_IN', b)
bc = Relationship(b, 'KNOWS', c)
ad = Relationship(a, 'KNOWS', d)
subgraph = a | ab | bc | ad
print(subgraph)
graph.create(subgraph)

<img src="images/py2neo_create_node_again.png">

<a id="ogm"></a>
## OGM

OGM 類似於 ORM，意為 Object Graph Mapping，這樣可以實現一個物件和 Node 的關聯，例如：

In [40]:
from py2neo.ogm import GraphObject, Property, RelatedTo, RelatedFrom

class Movie(GraphObject):
    __primarykey__ = 'title'
 
    title = Property()
    released = Property()
    actors = RelatedFrom('Person', 'ACTED_IN')
    directors = RelatedFrom('Person', 'DIRECTED')
    producers = RelatedFrom('Person', 'PRODUCED')

class Person(GraphObject):
    __primarykey__ = 'name'
 
    name = Property()
    born = Property()
    acted_in = RelatedTo('Movie')
    directed = RelatedTo('Movie')
    produced = RelatedTo('Movie')

In [42]:
from py2neo import Graph, Node, Relationship
from py2neo.ogm import GraphObject, Property, RelatedFrom

graph = Graph('http://127.0.0.1:7474', username='neo4j', password='42840667')
# graph = Graph("bolt://localhost:7687", auth=("neo4j", "42840667"))

class Foo(GraphObject):
    __primarykey__ = 'name'

    def __init__(self, name):
        self.name = name

    name = Property()
    age = Property()

class Biz(GraphObject):
    __primarykey__ = 'name'

    def __init__(self, name):
        self.name = name

    name = Property()
    age = Property()
    foos = RelatedFrom(Foo, "FOOS")

foo = Foo('foo')
biz = Biz('biz')
biz.foos.add(foo)

# This will work on the first run but fail on the second.
graph.push(biz)

In [43]:
from py2neo import Graph
from py2neo.ogm import GraphObject, Property

class Person(GraphObject):
    __primarykey__ = "name"

    name = Property()
    age = Property()
    location = Property()

# Query by property
person = Person.match(graph).where(age=21).first()

# Query by name
person = Person.match(graph, "Alice").first()
if person is not None:
    print(person)
    print(person.name)
    print(person.age)
    print(person.location)

In [44]:
if person is not None:
    print(person.__ogm__.node)

In [45]:
# Query by __primarykey__
alice = Person.match(graph,'Alice').first()

In [46]:
# Delete alice Object
if alice is not None:
    graph.delete(alice)

In [47]:
user_want_to_delete = Person.match(graph,'Mike').first()
if user_want_to_delete is not None:
    graph.delete(user_want_to_delete)

### Create node by Person class

In [48]:
user = Person()
user.name ="David"
user.age = 25
user.location = "New Taipei City"

graph.push(user)

<a id="class"></a>
## User Class

In [49]:
class Person(GraphObject):
    __primarykey__ = "name"

    name = Property()
    age = Property()
    location = Property()
    
    #knows=RelatedTo('Person','KNOWS')
    #known=RelatedFrom('Person','KNOWN')

    def __init__(self, name):
        self.name = name
    
    def find(self):
        user = self.match(graph, self.name).first()
        return user

    def register(self, age, location):
        if not self.find():
            user = Node('Person', name=self.name, age=age, location=location)
            #ad = Relationship(a, 'KNOWS', d)
            graph.create(user)
            return True
        else:
            return False

In [50]:
p = Person("Alice")
p.register(21, "台北")
p = Person("Bob")
p.register(22, "台中")
p = Person("Mike")
p.register(23, "高雄")

True

In [51]:
from py2neo import Graph, Node, Relationship, NodeMatcher, RelationshipMatcher

matcher = NodeMatcher(graph)
rmatcher = RelationshipMatcher(graph)

In [52]:
a = Person("Alice").__ogm__.node
b = Person("Bob").__ogm__.node
c = Person("Mike").__ogm__.node
ab =  Relationship(a, "ACTED_IN", b)
bc =  Relationship(b, "KNOWS", c)

graph.create(ab)

In [53]:
ndf = graph.run("MATCH (a:Person) RETURN a.name, a.age, a.location LIMIT 10").to_data_frame()
ndf

,a.name,a.age,a.location
0,David,25.0,New Taipei City
1,Alice,21.0,台北
2,Bob,22.0,台中
3,Mike,23.0,高雄
4,Alice,NaN,None
5,Bob,NaN,None


In [54]:
nodeList = list(matcher.match().where("_.name =~ 'B.*'").order_by("_.age").limit(4))
print(len(nodeList))
for i in nodeList:
    print(i, i["name"])

2
(_4:Person {age: 22, location: '\u53f0\u4e2d', name: 'Bob'}) Bob
(_7:Person {name: 'Bob'}) Bob


### DELETE Node

```
MATCH (n:`User name`) OPTIONAL MATCH (n)-[r]-() DELETE r,n
```

In [55]:
graph = Graph('http://127.0.0.1:7474', username='neo4j', password='42840667')
# graph = Graph("bolt://localhost:7687", auth=("neo4j", "42840667"))
graph.run("MATCH (n:`User`) OPTIONAL MATCH (n)-[r]-() DELETE r,n")
graph.run("MATCH (n:`Person`) OPTIONAL MATCH (n)-[r]-() DELETE r,n")
graph.run("MATCH (n:`Biz`) OPTIONAL MATCH (n)-[r]-() DELETE r,n")
graph.run("MATCH (n:`Foo`) OPTIONAL MATCH (n)-[r]-() DELETE r,n")

## LAB

Create 4 people with Node name `Person`, where:

- A: David
- B: Arnold
- C: Gracia
- D: Emily

David KNOWS Arnold, Arnold LIKES Gracia, David LOVES Emily, and given their age, David is 35 years old, Arnold is 28, Gracia is 24, Emily is 22. Please create Nodes and Relationships with Py2neo.

After you create Nodes, Relationships locally, push to Desktop Neo4j server.

In [56]:
from py2neo import Node, Relationship

graph = Graph('http://127.0.0.1:7474', username='neo4j', password='42840667')
# graph = Graph("bolt://localhost:7687", auth=("neo4j", "42840667"))

a = Node('Person', name='David')
b = Node('Person', name='Arnold')
c = Node('Person', name='Gracia')
d = Node('Person', name='Emily')

a['age'] = 35
b['age'] = 28
c['age'] = 24
d['age'] = 22

ab = Relationship(a, 'KNOWS', b)
bc = Relationship(b, 'LIKES', c)
ac = Relationship(a, 'KNOWS', c)
ad = Relationship(a, 'LOVES', d)

subgraph = ab | bc | ac | ad
print(subgraph)
graph.create(subgraph)

Subgraph({Node('Person', age=35, name='David'), Node('Person', age=24, name='Gracia'), Node('Person', age=28, name='Arnold'), Node('Person', age=22, name='Emily')}, {KNOWS(Node('Person', age=35, name='David'), Node('Person', age=24, name='Gracia')), LOVES(Node('Person', age=35, name='David'), Node('Person', age=22, name='Emily')), KNOWS(Node('Person', age=35, name='David'), Node('Person', age=28, name='Arnold')), LIKES(Node('Person', age=28, name='Arnold'), Node('Person', age=24, name='Gracia'))})


## Duplicate Nodes when running more than once?

<img src="images/duplicated_nodes_when_create.png">

<a id="merge"></a>
## MERGE AND UNIQUE Nodes

```
CREATE CONSTRAINT ON (p:Person)
ASSERT p.name IS UNIQUE
```

### Drop CONSTRAINT
```
graph.run("DROP CONSTRAINT ON (p:Person) ASSERT p.name IS UNIQUE")
```

In [ ]:
# graph.run("DROP CONSTRAINT ON (p:Person) ASSERT p.name IS UNIQUE")

### Create CONSTRAINT
```
graph.run("CREATE CONSTRAINT ON (p:Person) ASSERT p.name IS UNIQUE")
```

In [57]:
graph.run("CREATE CONSTRAINT ON (p:Person) ASSERT p.name IS UNIQUE")

In [58]:
from py2neo import Graph,Node,Relationship

a = Node('Person', name='David')
b = Node('Person', name='Arnold')
c = Node('Person', name='Gracia')
d = Node('Person', name='Emily')

a['age'] = 35
b['age'] = 28
c['age'] = 24
d['age'] = 22
a['location'] = "Taipei City" # adding property

a.__primarylabel__ = "Person"
a.__primarykey__ = "name"
b.__primarylabel__ = "Person"
b.__primarykey__ = "name"
c.__primarylabel__ = "Person"
c.__primarykey__ = "name"
d.__primarylabel__ = "Person"
d.__primarykey__ = "name"

ab = Relationship(a, 'KNOWS', b)
bc = Relationship(b, 'LIKES', c)
ac = Relationship(a, 'KNOWS', c)
ad = Relationship(a, 'LOVES', d)

transaction = graph.begin()
graph.merge(a)
graph.merge(b)
graph.merge(c)
graph.merge(d)
graph.merge(ab)
graph.merge(bc)
graph.merge(ac)
graph.merge(ad)
transaction.graph.push(a)

## Summary

When create Nodes on server, do consider CONSTRAINT on each Nodes, even different type of Nodes, I'll leave one question to you, what about create another type of Node, for example: Role

aa = Node('Role', name='Manager')
bb = Node('Role', name='Team Lead')

you may try add another CONSTRAINT like this:

```
CREATE CONSTRAINT ON (r:Role)
ASSERT r.name IS UNIQUE
```